In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json

In [204]:
zillow_url = "https://www.zillow.com/miami-fl/"

urls = [zillow_url + f"{str(page)}_p/" for page in range(1, 10, 1)]

headers = {
    'authority': 'www.zillow.com',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,es-US;q=0.8,es;q=0.7',
    'sec-ch-ua': '"Not/A)Brand";v="99", "Google Chrome";v="115", "Chromium";v="115"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
}

In [82]:
url1 = 'https://www.zillow.com/miami-fl/1_p/'

In [208]:
urls

['https://www.zillow.com/miami-fl/1_p/',
 'https://www.zillow.com/miami-fl/2_p/',
 'https://www.zillow.com/miami-fl/3_p/',
 'https://www.zillow.com/miami-fl/4_p/',
 'https://www.zillow.com/miami-fl/5_p/',
 'https://www.zillow.com/miami-fl/6_p/',
 'https://www.zillow.com/miami-fl/7_p/',
 'https://www.zillow.com/miami-fl/8_p/',
 'https://www.zillow.com/miami-fl/9_p/']

In [205]:
property_data = {}

In [209]:
for url in urls:
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.content)
    
    #get info on all listed on the page
    properties = soup.find_all("li", class_ = "ListItem-c11n-8-84-3__sc-10e22w8-0 StyledListCardWrapper-srp__sc-wtsrtn-0 iCyebE gTOWtl")
    
    #filter through each listing
    for prop in properties:
        print(prop)
        
        try:
            
            #property ID
            article_tag = prop.find("article", {"role" : "presentation"})
            property_id = article_tag.attrs["id"].split("_")[1]
            
        #place all data inside a dictionary for each property
        property_dict = {}
        
        #property type and location found inside a JSON format
        prop_info = prop.find("script", {"type" : "application/ld+json"}).string
        data = json.loads(prop_info)
        property_dict["property_type"] = data["@type"]
        property_dict["address"] = data["name"]
        property_dict["floor_size"] = data['floorSize']['value']
        property_dict["floor_size_value_type"] = data['floorSize']['@type']
        property_dict["street_address"] = data['address']['streetAddress']
        property_dict["locality"] = data['address']['addressLocality']
        property_dict["state"] = data['address']['addressRegion']
        property_dict["postal_code"] = data['address']['postalCode']
        property_dict["latitude"] = data['geo']['latitude']
        property_dict["longitude"] = data['geo']['longitude']
        
        # other useful information outside json format
        card_info = prop.find("div", class_ = "StyledPropertyCardDataWrapper-c11n-8-84-3__sc-1omp4c3-0 bKpguY property-card-data")
        
        property_dict["selling_price"] = card_info.find("span", class_="PropertyCardWrapper__StyledPriceLine-srp__sc-16e8gqd-1 iMKTKr").get_text(strip = True)
        property_dict["property_agency"] =  card_info.find("div", class_="StyledPropertyCardDataArea-c11n-8-84-3__sc-yipmu-0 jretvB").get_text(strip = True)
        #amenity info    
        amenities = card_info.find("ul", class_="StyledPropertyCardHomeDetailsList-c11n-8-84-3__sc-1xvdaej-0 eYPFID").find_all("li")
        property_dict["rooms"] = rooms = amenities[0].b.string
        property_dict["baths"] = baths = amenities[1].b.string
        property_dict["area"] = area = amenities[2].b.string
        property_dict["link"] = card_info.a.get("href")
        
        
        
    #append single property data 
    
    property_data[property_id] = property_dict

<li class="ListItem-c11n-8-84-3__sc-10e22w8-0 StyledListCardWrapper-srp__sc-wtsrtn-0 iCyebE gTOWtl"><div data-renderstrat="ssr" style="height:100%;display:grid"><div><script type="application/ld+json">{"@type":"SingleFamilyResidence","@context":"http://schema.org","name":"19800 SW 180th Ave #50506, Miami, FL 33187","floorSize":{"@type":"QuantitativeValue","@context":"http://schema.org","value":"1,690"},"address":{"@type":"PostalAddress","@context":"http://schema.org","streetAddress":"19800 SW 180th Ave #50506","addressLocality":"Miami","addressRegion":"FL","postalCode":"33187"},"geo":{"@type":"GeoCoordinates","@context":"http://schema.org","latitude":25.58159,"longitude":-80.48648},"url":"https://www.zillow.com/homedetails/19800-SW-180th-Ave-50506-Miami-FL-33187/2056542201_zpid/"}</script><article class="StyledPropertyCard-c11n-8-84-3__sc-jvwq6q-0 kbUUtf StyledPropertyCard-srp__sc-1o67r90-0 bdwyNr property-card list-card_not-saved" data-test="property-card" id="zpid_2056542201" role="p

AttributeError: 'NoneType' object has no attribute 'attrs'

In [206]:
property_data

{}

In [194]:
def save_data(title, data):
    with open(title, 'w', encoding = "utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent= 2)

In [201]:
save_data("zillow_data.json", property_data)